I used excel for splitting slach in columns like fuel types and etc by Extentions then power tools then start then split and custom by (/) and plit by rows
I used Sum and average in seats column because it has arithmetic operations
I will focus now in hanling units and converting them to a common format

In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
import re

# Load the dataset
df = pd.read_csv("Data/CCar.csv") 


In [2]:
# Check for duplicate rows
duplicate_count = df.duplicated().sum()

# Output result
print("Total number of duplicate rows:\n",duplicate_count)

Total number of duplicate rows:
 0


In [3]:
# Count missing values in each column
missing_values = df.isnull().sum()

# Display result
print(missing_values)

Company Names                0
Cars Names                   0
Engines                      0
Battery Capacity_CC          2
HorsePower_hp                0
Total Speed_km/h             0
Performance(0 - 100 )KM/H    6
Fuel Types                   0
Seats                        0
Torque_Nm                    1
Cars Prices_USD              0
dtype: int64


In [4]:
# Show data types of each column
print(df.dtypes)


Company Names                object
Cars Names                   object
Engines                      object
Battery Capacity_CC          object
HorsePower_hp                object
Total Speed_km/h             object
Performance(0 - 100 )KM/H    object
Fuel Types                   object
Seats                         int64
Torque_Nm                    object
Cars Prices_USD              object
dtype: object


In [5]:
#  Clean battery capacity values
def clean_battery_capacity(value):
    if pd.isna(value):
        return np.nan

    val = str(value).lower().replace(',', '').strip()

    # Exclude kWh entries
    if 'kwh' in val:
        return np.nan

    # Handle range like "1300-1500"
    if '-' in val:
        nums = re.findall(r'\d+\.?\d*', val)
        if len(nums) == 2:
            return (float(nums[0]) + float(nums[1])) / 2

    # Extract single cc value
    match = re.search(r'\d+\.?\d*', val.replace('cc', ''))
    return float(match.group()) if match else np.nan

#  Apply the cleaning function
df['Battery Capacity_CC'] = df['Battery Capacity_CC'].apply(clean_battery_capacity)

# Calculate the mean from all valid cc values (excluding only NaN and kWh)
realistic_mean = df['Battery Capacity_CC'].mean()

# Output the calculated mean
print(f"Realistic mean of Battery Capacity (cc): {realistic_mean:.2f}")
# Fill missing and kWh-derived NaN values with the realistic mean
df['Battery Capacity_CC'] = df['Battery Capacity_CC'].fillna(realistic_mean)



Realistic mean of Battery Capacity (cc): 2813.62


In [6]:
#fix horsepower,torque and prices columns

# Helper function to clean numeric values with optional unit removal and range handling
def clean_numeric_column(value, unit_keywords=None, allow_range=True):
    if pd.isna(value):
        return np.nan
    val = str(value).lower().replace(',', '').replace('$', '').strip()
    if unit_keywords:
        for keyword in unit_keywords:
            val = val.replace(keyword.lower(), '')
    if allow_range and '-' in val:
        nums = re.findall(r'\d+\.?\d*', val)
        if len(nums) == 2:
            return (float(nums[0]) + float(nums[1])) / 2
    match = re.search(r'\d+\.?\d*', val)
    return float(match.group()) if match else np.nan

# Clean specific columns
df['HorsePower_hp'] = df['HorsePower_hp'].apply(lambda x: clean_numeric_column(x, unit_keywords=['hp']))
df['Torque_Nm'] = df['Torque_Nm'].apply(lambda x: clean_numeric_column(x, unit_keywords=['nm']))
df['Cars Prices_USD'] = df['Cars Prices_USD'].apply(lambda x: clean_numeric_column(x))

# Impute missing values with mean for each column
for col in ['HorsePower_hp', 'Torque_Nm', 'Cars Prices_USD']:
    mean_val = df[col].mean()
    df[col] = df[col].fillna(mean_val)



In [7]:
#Fix Performance column
#  Rename the column to reflect its true content
df.rename(columns={'Performance(0 - 100 )KM/H': 'Acceleration_0_100_sec'}, inplace=True)

# Clean the values (remove 'sec', handle ranges, convert to float)
def clean_acceleration(value):
    if pd.isna(value):
        return np.nan
    val = str(value).lower().replace('sec', '').strip()
    if '-' in val:
        nums = re.findall(r'\d+\.?\d*', val)
        if len(nums) == 2:
            return (float(nums[0]) + float(nums[1])) / 2
    match = re.search(r'\d+\.?\d*', val)
    return float(match.group()) if match else np.nan

df['Acceleration_0_100_sec'] = df['Acceleration_0_100_sec'].apply(clean_acceleration)

# Fill missing values with the mean
mean_acc = df['Acceleration_0_100_sec'].mean()
df['Acceleration_0_100_sec'] = df['Acceleration_0_100_sec'].fillna(mean_acc)


In [8]:
# Clean and impute 'Total Speed_km/h'
def clean_total_speed(value):
    if pd.isna(value):
        return np.nan
    val = str(value).lower().replace('km/h', '').strip()
    if '-' in val:
        nums = re.findall(r'\d+\.?\d*', val)
        if len(nums) == 2:
            return (float(nums[0]) + float(nums[1])) / 2
    match = re.search(r'\d+\.?\d*', val)
    return float(match.group()) if match else np.nan

# Apply cleaning function
df['Total Speed_km/h'] = df['Total Speed_km/h'].apply(clean_total_speed)

# Fill missing values with the mean
mean_speed = df['Total Speed_km/h'].mean()
df['Total Speed_km/h'] = df['Total Speed_km/h'].fillna(mean_speed)


In [9]:
#Fix Engines column

# Function to extract engine displacement in Liters (e.g., 1.6L → 1.6)
def extract_displacement(engine_str):
    if pd.isna(engine_str):
        return np.nan
    match = re.search(r'(\d+\.?\d*)\s*[lL]', engine_str)
    return float(match.group(1)) if match else np.nan

# Function to extract engine configuration (V6, I4, BOXER-4)
def extract_engine_type(engine_str):
    if pd.isna(engine_str):
        return np.nan
    match = re.search(r'\b(V\d+|I\d+|INLINE-\d+|BOXER-\d+)\b', engine_str.upper())
    return match.group(1) if match else np.nan

# Function to classify powertrain: Electric, Hybrid, or Combustion
def classify_powertrain(engine_str):
    if pd.isna(engine_str):
        return "Unknown"
    val = engine_str.lower()
    if "electric" in val and "hybrid" not in val:
        return "Electric"
    elif "hybrid" in val or ("electric" in val and "+" in val):
        return "Hybrid"
    else:
        return "Combustion"

# Apply functions to create new columns

df['Power_Type'] = df['Engines'].apply(classify_powertrain)


In [10]:
# Fix Fuel Types column

# Initial cleanup: strip and title-case
df['Fuel Types'] = df['Fuel Types'].str.strip().str.title()

# First pass of cleaning known issues
fuel_clean_map = {
    'Plug In Hyrbrid': 'Plug-in Hybrid',
    'Hybrid (Petrol)': 'Hybrid',
    'Petrol (Hybrid)': 'Hybrid',
    'Diesel Hybrid': 'Hybrid',
    'Ev': 'Electric',
    'Electric Only': 'Electric',
    'Awd': np.nan,  # Not a fuel type
    'Hybrid ': 'Hybrid',
    'Diesel ': 'Diesel'
}
df['Fuel Types'] = df['Fuel Types'].replace(fuel_clean_map)

# Fix case inconsistency for "Plug-in Hybrid"
df['Fuel Types'] = df['Fuel Types'].replace({'Plug-In Hybrid': 'Plug-in Hybrid'})



In [11]:
# Fix Company Names column)

# Standardize capitalization and remove extra spaces
df['Company Names'] = df['Company Names'].str.strip().str.title()

# Correct known typos and variants
company_fix_map = {
    'Frrari': 'Ferrari',
    'Toyot': 'Toyota',
    'Tsla': 'Tesla',
    'Frd': 'Ford',
    'Mistubishi': 'Mitsubishi',
    'Mazda ': 'Mazda',  # Extra space
    'Kia': 'KIA',
    'Bmw': 'BMW',
    'Audi': 'Audi',
    'Mercedes': 'Mercedes',
    'Nissan': 'Nissan',
    'Volkswagen': 'Volkswagen',
    'Volvo': 'Volvo'
}

# Apply corrections
df['Company Names'] = df['Company Names'].replace(company_fix_map)



In [12]:
#determine columns
print(df.columns.tolist())


['Company Names', 'Cars Names', 'Engines', 'Battery Capacity_CC', 'HorsePower_hp', 'Total Speed_km/h', 'Acceleration_0_100_sec', 'Fuel Types', 'Seats', 'Torque_Nm', 'Cars Prices_USD', 'Power_Type']


In [13]:
df.drop(columns=['Engines', 'Power_Type'], inplace=True)


In [14]:
df.drop(columns=['Cars Names'], inplace=True)


In [15]:
# Identify numeric columns
numeric_cols = [
    'Battery Capacity_CC',
    'HorsePower_hp',
    'Total Speed_km/h',
    'Acceleration_0_100_sec',
    
    'Torque_Nm',
    'Cars Prices_USD'
]

# Calculate outlier counts before capping
outlier_counts = {}
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outlier_counts[col] = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
    # Apply IQR capping
    df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)

# Display outlier counts
print("Outlier counts per numeric column:")
for col, count in outlier_counts.items():
    print(f"{col}: {count}")


Outlier counts per numeric column:
Battery Capacity_CC: 124
HorsePower_hp: 55
Total Speed_km/h: 36
Acceleration_0_100_sec: 34
Torque_Nm: 48
Cars Prices_USD: 156


In [16]:
# Detect outliers in 'Seats' only
Q1 = df['Seats'].quantile(0.25)
Q3 = df['Seats'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Calculate mean of non-outliers (and round to int)
non_outlier_mean = int(df[(df['Seats'] >= lower_bound) & (df['Seats'] <= upper_bound)]['Seats'].mean())

# Replace outliers with the rounded mean
df.loc[(df['Seats'] < lower_bound) | (df['Seats'] > upper_bound), 'Seats'] = non_outlier_mean


In [17]:
# Encode (Fuel Types) using Label Encoding
#from sklearn.preprocessing import LabelEncoder

#df['Fuel Types'] = LabelEncoder().fit_transform(df['Fuel Types'])

In [18]:
df = df.round(1)


In [19]:
df.to_csv("Data/PreprocessedCar.csv", index=False)


In [20]:
print(df['Seats'].unique())
print(df['Seats'].value_counts())


[5]
Seats
5    1531
Name: count, dtype: int64
